In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ✅ Correct dataset path
data_dir = r"C:\Users\USER\OneDrive\Desktop\skillup\task_04\leapGestRecog"

X = []
y = []
labels_dict = {}  # Map gesture names to numeric IDs
label_id = 0

# Loop through subjects (00, 01, 02, ...)
for subject in sorted(os.listdir(data_dir)):
    subject_path = os.path.join(data_dir, subject)
    if not os.path.isdir(subject_path):
        continue
    
    # Loop through gesture folders (01_palm, 02_l, ...)
    for gesture_folder in sorted(os.listdir(subject_path)):
        gesture_path = os.path.join(subject_path, gesture_folder)
        if not os.path.isdir(gesture_path):
            continue
        
        # Assign numeric ID if new gesture
        if gesture_folder not in labels_dict:
            labels_dict[gesture_folder] = label_id
            label_id += 1
        
        # Load images
        for img_file in os.listdir(gesture_path):
            img_path = os.path.join(gesture_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (64,64))  # resize for CNN
            X.append(img)
            y.append(labels_dict[gesture_folder])

# Convert to arrays
X = np.array(X).reshape(-1, 64, 64, 1) / 255.0
y = to_categorical(y, num_classes=len(labels_dict))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Dataset loaded successfully!")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Gesture Labels Mapping:", labels_dict)


✅ Dataset loaded successfully!
X shape: (20000, 64, 64, 1)
y shape: (20000, 10)
Gesture Labels Mapping: {'01_palm': 0, '02_l': 1, '03_fist': 2, '04_fist_moved': 3, '05_thumb': 4, '06_index': 5, '07_ok': 6, '08_palm_moved': 7, '09_c': 8, '10_down': 9}


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Number of gesture classes
num_classes = y.shape[1]

# Build CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=15,
                    batch_size=32)

# Save the trained model
model.save("hand_gesture_model.h5")

print("✅ Model trained and saved as hand_gesture_model.h5")


C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 56s 104ms/step - accuracy: 0.8947 - loss: 0.3181 - val_accuracy: 0.9985 - val_loss: 0.0059
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 52s 103ms/step - accuracy: 0.9919 - loss: 0.0258 - val_accuracy: 0.9998 - val_loss: 0.0010
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 52s 104ms/step - accuracy: 0.9937 - loss: 0.0191 - val_accuracy: 0.9990 - val_loss: 0.0034
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 105ms/step - accuracy: 0.9958 - loss: 0.0123 - val_accuracy: 0.9995 - val_loss: 0.0017
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 110ms/step - accuracy: 0.9974 - loss: 0.0078 - val_accuracy: 1.0000 - val_loss: 5.9129e-05
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 55s 111ms/step - accuracy: 0.9969 - loss: 0.0102 - val_accuracy: 0.9995 - val_loss: 0.0019
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 54s 108ms/step - accuracy: 0.9983 - loss: 0.0070 - val_accuracy: 0.9995 - val_loss: 0.0012
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 57s 113ms/step - accuracy: 0.9977 - los

✅ Model trained and saved as hand_gesture_model.h5


In [4]:
import json

# Save gesture label mapping
with open("gesture_labels.json", "w") as f:
    json.dump(labels_dict, f)

print("✅ Gesture labels saved in gesture_labels.json")


✅ Gesture labels saved in gesture_labels.json


In [6]:
import cv2
import numpy as np
import json
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("hand_gesture_model.h5")

# Load gesture labels
with open("gesture_labels.json", "r") as f:
    labels_dict = json.load(f)

# Reverse mapping: ID -> gesture name
id_to_label = {v: k for k, v in labels_dict.items()}

# Open webcam
cap = cv2.VideoCapture(0)

print("✅ Webcam started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize to match training input
    roi = cv2.resize(gray, (64,64))
    roi = roi.reshape(1,64,64,1) / 255.0

    # Prediction
    prediction = model.predict(roi)
    class_id = np.argmax(prediction)
    label = id_to_label[class_id]

    # Show result on frame
    cv2.putText(frame, f"Prediction: {label}", (10,50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Hand Gesture Recognition", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


✅ Webcam started. Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━